In [2]:
import pandas as pd
import pickle
import numpy as np
import time 

time: 162 ms (started: 2022-06-01 09:44:13 +00:00)


In [3]:
def save_to_pickle(pkl_path,dict_obj):
    file = open(pkl_path, "wb")
    pickle.dump(dict_obj, file)
    file.close()

time: 462 µs (started: 2022-06-01 09:44:13 +00:00)


In [4]:
def map_feature_label(target_path,dict_csv_path):
    all_feature=pd.read_csv(dict_csv_path)
    feature_label_divtionary={k:v for k,v in zip(all_feature['feature_name'],all_feature['label'])}
    item_feature=pd.read_csv(target_path)
    item_feature['feature_category_id'] = item_feature['feature_category_id'].apply(str)
    item_feature['feature_value_id'] = item_feature['feature_value_id'].apply(str)
    item_feature['feature_name']=item_feature["feature_category_id"] + "_" + item_feature["feature_value_id"]
    item_feature['label']=[feature_label_divtionary[i] for i in item_feature['feature_name']]
    item_feature['label'].apply(int)
    return item_feature

time: 555 µs (started: 2022-06-01 09:44:14 +00:00)


# 計算item_category+item_value之出現次數

In [5]:
item_feature=pd.read_csv("../../recsys2022/item_features.csv")#path to item_features.csv
# item_feature.columns
all_feature=item_feature.groupby(['feature_category_id','feature_value_id']).count()
# all_feature
all_feature.to_csv('__feature.csv')
all_feature=pd.read_csv('__feature.csv')
all_feature

,feature_category_id,feature_value_id,item_id
0,1,60,1
1,1,143,10
2,1,358,2
3,1,461,924
4,1,517,2
...,...,...,...
899,72,751,55
900,72,829,405
901,72,883,65
902,73,91,2381


time: 77.4 ms (started: 2022-06-01 09:44:15 +00:00)


# Data preprocessing
- item=>vector
- session=>vector
## feature encoding

In [6]:
all_feature=all_feature.drop(columns=['item_id'])

all_feature['feature_category_id'] = all_feature['feature_category_id'].apply(str)
all_feature['feature_value_id'] = all_feature['feature_value_id'].apply(str)
all_feature['feature_name']=all_feature["feature_category_id"] + "_" + all_feature["feature_value_id"]
all_feature['label']=[i for i in range(all_feature.shape[0])]
# all_feature=all_feature.drop(columns=['feature_category_id','feature_value_id'])
all_feature.head(3)

,feature_category_id,feature_value_id,feature_name,label
0,1,60,1_60,0
1,1,143,1_143,1
2,1,358,1_358,2


time: 5.64 ms (started: 2022-06-01 09:44:22 +00:00)


In [7]:
#all_feature.to_csv('all_feature_encoding.csv') #path to save

time: 204 µs (started: 2022-06-01 09:44:22 +00:00)


## item==>vetor
### 1. mapping

In [8]:
item_feature=map_feature_label('../../recsys2022/item_features.csv','../datasets/all_feature_encoding.csv')
item_feature["frequence"]=np.ones((len(item_feature),1))
item_feature

,item_id,feature_category_id,feature_value_id,feature_name,label,frequence
0,2,56,365,56_365,637,1.0
1,2,62,801,62_801,720,1.0
2,2,68,351,68_351,796,1.0
3,2,33,802,33_802,378,1.0
4,2,72,75,72_75,878,1.0
...,...,...,...,...,...,...
471746,28143,68,351,68_351,796,1.0
471747,28143,55,390,55_390,578,1.0
471748,28143,11,109,11_109,131,1.0
471749,28143,73,91,73_91,902,1.0


time: 397 ms (started: 2022-06-01 09:44:24 +00:00)


In [9]:
#pivot table
item_feature_p = item_feature.pivot_table(index="item_id", columns="feature_name",values="frequence")
item_feature_p = (item_feature_p.fillna(0))
item_feature_p.head()

feature_name,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,7_837,7_838,8_157,8_323,8_420,8_636,9_175,9_178,9_190,9_369
item_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 366 ms (started: 2022-06-01 09:44:24 +00:00)


In [10]:
item_feature_p.to_csv("item_freq_vec.csv")#path to save

time: 4.84 s (started: 2022-06-01 09:44:26 +00:00)


## train_sessions => vector

In [11]:
def items2vec(_df_item,_session_items):
    session_vec = np.sum(_df_item.loc[pd.Index(_session_items)]).values
    return session_vec 

time: 281 µs (started: 2022-06-01 09:44:31 +00:00)


In [12]:
def sessions2vec(item_vec_df,session_path,save_path):
    '''
    input 1 dataframe with col name :session_id,item_id
    input 2 path of session.csv
    imput 3 path you wanna save will save to pkl
    '''
    #item_vec裡面需要有col:session_id,item_id
    s_t = time.time()
    _df_session = pd.read_csv(session_path)
    _df_item = item_vec_df
    pre_sid = None
    session_dic={}
    item_buf=[]
    for row in _df_session.iterrows():#tuple of row value,row[0]=index,row[1]=values
        if pre_sid!=row[1].session_id:#check if session id matches
            if (item_buf):#if value in buf
                session_dic[pre_sid] = (items2vec(_df_item,item_buf))#calculate freq vec for session
            pre_sid = row[1].session_id# move to next session
            item_buf=[]
        item_buf.append(row[1].item_id)
    session_dic[pre_sid] = (items2vec(_df_item,item_buf))#saving last session vec 
    e_t=time.time()
    save_to_pickle(save_path+".pkl",session_dic)
    print("total session: ",len(session_dic)," time usage: ",e_t-s_t," sec")

time: 3.65 ms (started: 2022-06-01 09:44:31 +00:00)


In [ ]:
sessions2vec(item_feature_p,"../../recsys2022/train_sessions.csv","train_session")

## test session ==> vector

In [ ]:
sessions2vec(item_feature_p,'../../recsys2022/test_leaderboard_sessions.csv',"test_leader_session")

In [ ]:
sessions2vec(item_feature_p,'../../recsys2022/train_purchases.csv',"purchases_session")

In [21]:
sessions2vec(item_feature_p,'../datasets/train_last_05m.csv',"../datasets/train_session_last_05m")

total session:  47320  time usage:  82.71393537521362  sec
time: 1min 23s (started: 2022-06-01 16:50:08 +00:00)


In [22]:
sessions2vec(item_feature_p,'../datasets/purchases_last_05m.csv',"../datasets/purchases_session_last_05m")

total session:  47320  time usage:  71.12630128860474  sec
time: 1min 11s (started: 2022-06-01 16:51:31 +00:00)


In [23]:
sessions2vec(item_feature_p,'../datasets/train_purchases_last_05m.csv',"../datasets/train_purchases_session_last_05m")

total session:  47320  time usage:  86.05093359947205  sec
time: 1min 26s (started: 2022-06-01 16:52:43 +00:00)
